<a href="https://colab.research.google.com/github/dinithprimal/DeepLearning_Tutorials/blob/main/EX21_Optimize_Tensorflow_Pipeline_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.15.0'

In [22]:
class FileDataset(tf.data.Dataset):
  def read_files_in_batches(num_samples):
    # Open file
    time.sleep(0.03)
    for sample_idx in range(num_samples):
      time.sleep(0.015)
      yield (sample_idx,)

  def __new__(self, num_samples=3):
    return tf.data.Dataset.from_generator(
        self.read_files_in_batches,
        output_signature=tf.TensorSpec(shape = (1,), dtype = tf.int64),
        args=(num_samples,)
    )

In [23]:
def benchmark(dataset, num_epochs=2):
  for epoch_num in range(num_epochs):
    for sample in dataset:
      time.sleep(0.01)

In [24]:
%%timeit
benchmark(FileDataset())

293 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##Prefetch API

In [25]:
%%timeit
benchmark(FileDataset().prefetch(1))

256 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

277 ms ± 21.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cache API

In [27]:
dataset = tf.data.Dataset.range(5)

for d in dataset:
  print(d.numpy())

0
1
2
3
4


In [28]:
dataset = dataset.map(lambda x: x**2)
for d in dataset:
  print(d.numpy())

0
1
4
9
16


In [29]:
dataset = dataset.cache()

for d in dataset.as_numpy_iterator():
  print(d)

0
1
4
9
16


In [30]:
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [31]:
list(dataset.as_numpy_iterator())    # reading this from cache

[0, 1, 4, 9, 16]

In [33]:
def mapped_function(s):
  tf.py_function(lambda: time.sleep(0.03), [], ())
  return s

In [34]:
FileDataset().map(mapped_function)

<_MapDataset element_spec=TensorSpec(shape=(1,), dtype=tf.int64, name=None)>

In [35]:
%%timeit -n1 -r1

benchmark(FileDataset().map(mapped_function), 5)

1.12 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [36]:
%%timeit -n1 -r1

benchmark(FileDataset().map(mapped_function).cache(), 5)

557 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
